# Redes Neuronales
## Fashion detector 

In [ ]:
# de python, para especificar rutas de archivos y directorios
from pathlib import Path

# lib para trabajar con arrays
import numpy as np
import pandas as pd

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para construir y entrenar redes neuronales, y que además tiene utilidades para leer sets de
# imágenes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
CATEGORIAS = "T-shirt/top" , "Trouser" , "Pullover" , "Dress" , "Coat" , "Sandal" , "Shirt" , "Sneaker" , "Bag" , "Ankle boot"
##TRAIN_DIR = Path('./')
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

### 1) Análisis exploratorio sobre el conjunto de datos.  
Este es un dataset de 70000 imágenes en blanco y negro de 28x28 pixeles, de 10 categorías de prendas, divididas en un set de train con 60000 imágenes y otro de test de 10000. 

In [ ]:
print('Cantidad y tamaño de imágenes de train:')
x_train.shape  

In [ ]:
print('Cantidad y tamaño de imágenes de test:')
x_test.shape

In [ ]:
# diccionario de tipos que contiene:
# clave (salida): [nombre prenda, cantidad train, cantidad test]
CLASES = {
    0: ["T-shirt/top", 0, 0],
    1: ["Trouser", 0, 0],
    2: ["Pullover", 0, 0],
    3: ["Dress", 0, 0],
    4: ["Coat", 0, 0],
    5: ["Sandal", 0, 0],
    6: ["Shirt", 0, 0],
    7: ["Sneaker", 0, 0],
    8: ["Bag", 0, 0],
    9: ["Ankle boot", 0, 0]
}

Las categorías de prendas son las siguientes:

In [ ]:
for clave in CLASES:
    print(CLASES[clave][0])

Ejemplos de la imágenes sin modificaciones:

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(nombres[y_train[i]])
plt.show()

A partir de los gráficos siguientes, podemos notar que el set de datos está completamente balanceado tanto en train como test. En el set de train, hay 6000 imágenes de cada tipo de prenda, mientras que en el set de test hay 1000 imágenes de cada uno.

In [ ]:
cantidades = []
nombres = []

# función para contar y graficar la cantidad de prendas por tipo
def distribucion(salidas, posicion, titulo=''):
    for salida in salidas:
        CLASES[salida][posicion] += 1
        
    for clave in CLASES:
        cantidades.append(CLASES[clave][posicion])
        nombres.append(CLASES[clave][0])
    display(titulo)
    plt.pie(cantidades, labels=nombres, autopct="%0.1f %%")

In [ ]:
distribucion(y_train, 1, 'Distribución de train')

In [ ]:
distribucion(y_test, 2, 'Distribución de test')

### 2) Machine Learning. 

##### Reescalar imágenes  
En primera instancia reescalamos los valores de las imágenes, tanto en test como en train. Esto se puede comprobar en los siguientes gráficos que muestran el rango de valores que posee una imagen del dataset antes y después de reescalar.

In [ ]:
# reescalamos los valores de las imágenes
x_train_r = x_train/ 255.0
y_test_r = x_test / 255.0

In [ ]:
print('Antes de escalar:')
plt.figure()
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
print('Después de escalar:')
plt.figure()
plt.imshow(x_train_r[0])
plt.colorbar()
plt.grid(False)
plt.show()

##### Modificar el tamaño de las imágenes 
En cuanto al tamaño de las imágenes, optamos por NO modificarlo, debido a que ya es lo suficientemente pequeño como para entrenar sin demorar demasiado.